## KV Store
### Single Key, Single Value KVstore 

Let's create a key-value store with the keyword "device", which is used for single machine training. Notice that if you would like to create a key-value store for distributed training on multiple machines, you will use the keyword "dist". 

In [1]:
from mxnet import cpu, kv, np
kv = kv.create('device')

First, let's initialize a $(key, value)$ pair, which is an $(int, ndarray)$ pair, into the store, and pull the value out to the ndarray $\mathbf{A}$. From now on, we assume the shape of the ndarray will be $(2, 3)$ in our example.

In [2]:
shape = (2, 3)
kv.init(3, np.ones(shape)* (-2))
A = np.zeros(shape)
kv.pull(3, out = A)
A

array([[-2., -2., -2.],
       [-2., -2., -2.]])

For any key that has been initialized, we can always push a new value with the same shape to that key:

In [3]:
kv.push(3, np.ones(shape)* (-7))
kv.pull(3, out = A)
A

array([[-7., -7., -7.],
       [-7., -7., -7.]])

### Single Key, Multiple Values KVstore 

Furthermore, we can push multiple values (with the same shape) into the same key. In this case, the KVstore will first sum up the given list of values and then push the aggregated value to the key. In the following example, we will demonstrate pushing a list of values on CPU.

In [5]:
contexts = [cpu(i) for i in range(3)]
B = [np.ones(shape, ctx = ctx) for ctx in contexts]
kv.push(3, B)
kv.pull(3, out = A)

# A has a shape (2,3), while B has a 3 x (2,3) on each CPU, then a sums along CPUs
A

array([[3., 3., 3.],
       [3., 3., 3.]])

### Multiple $(key, value)$ Pairs KVstore 

All operations introduced above are only involved a single key. KVstore also provides an interface for a list of $(key, value)$ pairs. Here, we will demonstrate two examples, one on a single device, the other on multiple devices.

First, let's create a local KVstore a single device.

In [6]:
keys = [7, 11, 13, 17]
values = [np.ones(shape) * (-3)] * len(keys)
kv.init(keys, values)

C = [np.zeros(shape)] * len(keys)
kv.pull(keys, out = C)
C[0] # Output the value of the first key

[[-3. -3. -3.]
 [-3. -3. -3.]]


Next, we can also execute the similar operations on multiple devices. Suppose that each key is corresponding to a machine with 3 CPUs. In this case, each machine will aggregate its value and push it to its key.

In [23]:
D = [[np.ones(shape, ctx = ctx) * (-0.1) for ctx in contexts]] * len(keys)
print("D[0] : ", D[0])

kv.push(keys, D)
kv.pull(keys, out = C)
C[0] # Output the value of the first key

D[0] :  [array([[-0.1, -0.1, -0.1],
       [-0.1, -0.1, -0.1]]), array([[-0.1, -0.1, -0.1],
       [-0.1, -0.1, -0.1]]), array([[-0.1, -0.1, -0.1],
       [-0.1, -0.1, -0.1]])]


array([[-0.3, -0.3, -0.3],
       [-0.3, -0.3, -0.3]])

### Updater

At each push, KVstore combines the pushed value with the value stored using an updater. The default updater is ASSIGN, which is to assign the value to the key. We can replace the default updater and redefine how data is merged by `_set_updater`.

In [9]:
def update(key, input, stored):
    print("update on key: %d" % key)
    stored += input * 0.01
    
kv._set_updater(update)
kv.pull(3, out = A)
A # Since we haven't called "push", the value of A doesn't change


kv.push(3, mx.nd.ones(shape) * 5)
kv.pull(3, out = A)
print(A.asnumpy()) # We called push through the new updater with an input equals to 5

array([[3., 3., 3.],
       [3., 3., 3.]])